In [31]:
import os
import cv2
import keras
import random
import numpy as np
import tensorflow as tf
from os import environ

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.version)
print(tf.config.list_physical_devices('GPU'))

<module 'tensorflow._api.v2.version' from '/opt/homebrew/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>
[]


In [32]:

# For reproducible results
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


In [33]:

# Constants
val_size = 0.2
data_path_healthy = '/Users/sca/Downloads/healthy'
data_path_unhealthy = '/Users/sca/Downloads/unhealthy'

In [34]:
# Function to load images from folder and convert to numpy array
def load_images(path):
    files = os.listdir(path)
    images = []
    
    for file in files:
        file_path = os.path.join(path, file)
        image = cv2.imread(file_path)
        
        if image is not None:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_rgb = image.astype(np.float32)
            images.append(image_rgb)
        else:
            print(f"Unable to load image: {file}")

    return np.array(images)

In [35]:
# Read and preprocess the data
plants_healthy = load_images(data_path_healthy)
plants_unhealthy = load_images(data_path_unhealthy)

# Merge the data and create labels
X = np.concatenate([plants_healthy, plants_unhealthy], axis=0)
y = np.concatenate([np.zeros(len(plants_healthy)), np.ones(len(plants_unhealthy))], axis=0)

# Split the data into train and validation sets and normalize the images
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=seed, test_size=val_size, stratify=y)
X_train, X_val = X_train / 255.0, X_val / 255.0

Unable to load image: .DS_Store


In [36]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train, seed=seed)

In [37]:

class EnhancedModel:
    def __init__(self, input_shape=(96, 96, 3)):
        self.model = Sequential()

        # Convolutional Block 1
        self.model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        self.model.add(BatchNormalization())
        self.model.add(LeakyReLU())
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        # Convolutional Block 2
        self.model.add(Conv2D(64, (3, 3), padding='same'))
        self.model.add(BatchNormalization())
        self.model.add(LeakyReLU())
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.3))

        # Additional Convolutional Blocks with increasing filters
        # Adjust the number of blocks, filter sizes, and dropout rates as necessary

        # Global Average Pooling
        self.model.add(GlobalAveragePooling2D())

        # Dense Layer before Output
        self.model.add(Dense(512))
        self.model.add(BatchNormalization())
        self.model.add(LeakyReLU())
        self.model.add(Dropout(0.4))

        # Output Layer
        self.model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

        # Compile model
        self.model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

    def predict(self, X):
        return self.model.predict(X)
    
    def evaluate(self, X, y):
        loss, accuracy = self.model.evaluate(X, y)
        print(f"Test Accuracy: {accuracy*100:.2f}%")


In [38]:
# Example usage of the class
if __name__ == "__main__":
    model = EnhancedModel()
    model.model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
    predictions = model.predict(X_val)
    predictions = predictions.flatten()
    model.evaluate(X_val, y_val)





Epoch 1/10
125/125 [==============================] - 14s 109ms/step - loss: 0.6598 - accuracy: 0.6600 - val_loss: 0.7330 - val_accuracy: 0.6204
Epoch 2/10
125/125 [==============================] - 15s 118ms/step - loss: 0.6008 - accuracy: 0.7007 - val_loss: 1.5099 - val_accuracy: 0.6204
Epoch 3/10
125/125 [==============================] - 14s 109ms/step - loss: 0.5624 - accuracy: 0.7220 - val_loss: 2.1653 - val_accuracy: 0.6204
Epoch 4/10
125/125 [==============================] - 15s 117ms/step - loss: 0.5374 - accuracy: 0.7375 - val_loss: 0.5933 - val_accuracy: 0.6803
Epoch 5/10
125/125 [==============================] - 15s 119ms/step - loss: 0.5375 - accuracy: 0.7408 - val_loss: 0.5308 - val_accuracy: 0.7363
Epoch 6/10
125/125 [==============================] - 14s 115ms/step - loss: 0.5171 - accuracy: 0.7525 - val_loss: 0.6444 - val_accuracy: 0.6643
Epoch 7/10
125/125 [==============================] - 14s 114ms/step - loss: 0.4975 - accuracy: 0.7600 - val_loss: 1.2972 - val_ac

32/32 [==============================] - 4s 107ms/step - loss: 0.5969 - accuracy: 0.7193
Test Accuracy: 71.93%


In [40]:
model.model.save('/Users/sca/Desktop/AN2DL/ANN-DL-Competition1/SubmissionModel.keras')